In [1]:
import numpy as np
import scipy as sp
import matplotlib
from matplotlib import pyplot as plt
%matplotlib notebook
from tqdm import tqdm_notebook as tqdm
import tabulate
from IPython.display import display, HTML, Markdown
from scipy.spatial.distance import cdist as dist
from notebook_functions import open_file_dialog, read_h5_data, blue, orange

############# SCRIPT PARAMETERS #############

data_set = 'vdc_8'

#############################################

path_to_file = open_file_dialog()

parameters = {'File': path_to_file,
              'Dataset': data_set
             }

# display(Markdown('### <center> Notebook parameters </center>'))
display(HTML('<center>' + tabulate.tabulate(parameters.items(), tablefmt='html') + '</center>'))

File,D:/Data/1.8 um design small vernier/small_vernier_tx_data.h5
Dataset,vdc_8


In [2]:
## load data from file
if path_to_file is not None:
    data = read_h5_data(path_to_file, data_set, add_zero_freq=True, minimal=False)

In [13]:
freqs = data['freqs']
x = data['x']
x_mem = data['x_mem']
x_ch = data['x_ch']

x_var = np.var(np.abs(x), axis=0)
x_mem_var = np.var(np.abs(x_mem), axis=0)
x_ch_var = np.var(np.abs(x_ch), axis=0)

In [12]:
def surface_plot(meminfo, freqs, f, cmap='RdBu_r', ax=None, update=False):
    
    surfs = interpolate_surfaces(meminfo, freqs, f)
    
    if not update:
        
        for xig, yig, zi in surfs:
            im = ax.pcolormesh(xig, yig, zi, cmap=cmap, shading='gouraud')

        zmax = []
        for mem in meminfo:

            z = mem['x']
            zmax.append(np.max(np.abs(np.real(z))))
        
        zmax = max(zmax)

        ax.set_axis_off()
        ax.set_aspect('equal')
        
    else:
        
        for im in [a for a in ax.get_children() if type(a) is mpl_toolkits.mplot3d.art3d.Poly3DCollection]:
            im.remove()
            
        for xig, yig, zi in surfs:
            im_new = ax.pcolormesh(xig, yig, zi, cmap=cmap, shading='gouraud')
            
            
def surface_variance_plot(v, freqs, f, ax=None, lines=None, update=False):

    i = np.argmin(np.abs(f - freqs))
    
    if not update:

        v_line, = ax.plot(freqs/1e6, v, color=blue)
        v_marker, = ax.plot(freqs[i]/1e6, v[i], color=blue, marker='o')

        f_line = ax.axvline(x=freqs[i]/1e6, ls=':')

        lines = [v_line, v_marker, f_line]
        
        ax.set_xlim(0, freqs.max()/1e6)
        ax.set_xlabel('Frequency (MHz)')
        ax.set_ylabel('Displacement variance')
        
        ax.grid('on')
        
        return lines
    
    else:

        v_line, v_marker, f_line = lines
        
        v_marker.set_data(freqs[i]/1e6, v[i])
        f_line.set_xdata(freqs[i]/1e6)


def onclick(event):
    
    if event.inaxes in [ax1l, ax1r]:
        
        f = event.xdata*1e6
        
        surface_variance_plot(x_var, freqs, ax=ax1, f=f, lines=lines, update=True)
        surface_plot(header, f=f, ax=ax2, cmap='RdBu_r', update=True)

        
fig = plt.figure(figsize=(9,9))
gs = GridSpec(2,1)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
cid = fig.canvas.mpl_connect('button_press_event', onclick)
lines = surface_variance_plot(x_var, freqs, ax=ax1, f=1e6, lines=None, update=False)
# surface_plot(header, f=10e6, ax=ax2, cmap='RdBu_r', update=False)
plt.tight_layout()


<IPython.core.display.Javascript object>

NameError: name 'header' is not defined